In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import torch

t_c = torch.tensor([0.5,14.0,15.0,28.0,11.0,8.0,3.0,-4.0,6.0,13.0,21.0])
t_u = torch.tensor([35.7,55.9,58.2,81.9,56.3,48.9,33.9,21.8,48.4,60.4,68.4])

t_un = 0.1 * t_u

n_samples = t_u.shape[0]
n_val = int(0.2*n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices=shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices,val_indices

(tensor([ 3,  5,  4,  8,  2,  7,  1,  0, 10]), tensor([9, 6]))

In [3]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1*train_t_u
val_t_un = 0.1*val_t_u

In [5]:
def model(t_u,w,b):
  return w * t_u + b

def loss_fn(t_p,t_c):
  squared_diffs = (t_p - t_c)**2
  return squared_diffs.mean()

def training_loop(n_epochs,optimizer,params,train_t_u,val_t_u,
                  train_t_c,val_t_c):
  for epoch in range(1,n_epochs+1):
    train_t_p = model(train_t_u,*params)
    train_loss = loss_fn(train_t_p,train_t_c)
    
    val_t_p = model(val_t_u,*params)
    val_loss = loss_fn(val_t_p,val_t_c)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch<=3 or epoch%500==0:
      print(f"Epoch {epoch}, Training Loss {train_loss.item():.4f},"f" Validation loss{val_loss.item():.4f}")
  return params

In [7]:
import torch.optim as optim
params = torch.tensor([1.0,0.0],requires_grad=True)
learning_rate = 1e-2
optimizer = optim.SGD([params],lr=learning_rate)

training_loop(
    n_epochs=3000,
    optimizer=optimizer,
    params=params,
    train_t_u=train_t_un,
    val_t_u=val_t_un,
    train_t_c=train_t_c,
    val_t_c=val_t_c
)

Epoch 1, Training Loss 92.8238, Validation loss24.2968
Epoch 2, Training Loss 41.4546, Validation loss7.2096
Epoch 3, Training Loss 34.3515, Validation loss10.2757
Epoch 500, Training Loss 8.0515, Validation loss2.1452
Epoch 1000, Training Loss 3.4542, Validation loss2.9397
Epoch 1500, Training Loss 2.6258, Validation loss4.3778
Epoch 2000, Training Loss 2.4765, Validation loss5.1867
Epoch 2500, Training Loss 2.4496, Validation loss5.5659
Epoch 3000, Training Loss 2.4448, Validation loss5.7332


tensor([  5.5790, -18.4106], requires_grad=True)